In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import TomekLinks, RandomUnderSampler
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

In [ ]:
# Carregando a base Titanic
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

In [ ]:
# Tratamento inicial dos dados
df = df[['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked']]
df.dropna(subset=['Embarked'], inplace=True)
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

# Separando features e labels
X = df.drop(columns=['Survived'])
y = df['Survived']

In [ ]:
# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Criando dados com valores ausentes para simulacao
X_com_ausentes = X.copy()
X_com_ausentes.loc[np.random.randint(0, X.shape[0], 20), 'Idade'] = np.nan

# Imputacao por Media
imputador_media = SimpleImputer(strategy='mean')
X_imputado_media = imputador_media.fit_transform(X_com_ausentes)

# Imputacao por Moda
imputador_moda = SimpleImputer(strategy='most_frequent')
X_imputado_moda = imputador_moda.fit_transform(X_com_ausentes)

# Imputacao por KNN
imputador_knn = KNNImputer(n_neighbors=3)
X_imputado_knn = imputador_knn.fit_transform(X_com_ausentes)

# Avaliacao
modelos = {
    "Media": X_imputado_media,
    "Moda": X_imputado_moda,
    "KNN": X_imputado_knn
}

for metodo, X_imputado in modelos.items():
    modelo = RandomForestClassifier().fit(X_imputado, y)
    y_previsto = modelo.predict(X_teste)
    print(f"{metodo} - Acuracia:", accuracy_score(y_teste, y_previsto))
